In [23]:
# "Location - Nose, Eyelid, Nail, Lip, Genitalia, or Acral (L)" PE 100%/A&P 100%/A 100%

# Is there a cancerous tumor on the: Nose, Eyelid, Nail, Lip, Genitalia, or Acral (Y/N)

import os
import pandas as pd
import requests
import base64

#using azure
API_KEY = "2DWhd5NlWo6U8lpUBSWgyCuCL1bgNUAlQyv4sF2Oq6TsNxszq97NJQQJ99AKACHrzpqXJ3w3AAABACOGzNKu"
ENDPOINT = "https://rg-01-hippa-standalone-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

def call_gpt(query):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    
    # Payload for the request
    payload = {
    "messages" : [
            {"role": "system", "content": "Answer the question based only on the following context, which can include text and tables: "},
            {"role": "user", "content": f"{query}"},
        ],
      "temperature": 0.7,
      "top_p": 0.95,
      "max_tokens": 800
    }
    
    
    # Send request
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        print("Response Content:", response.text)
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    #print(response.json())
    #return response.json().choices[0].message.content
    return response.json()["choices"][0]["message"]["content"]

In [24]:
# Use full Excel sheet
import pandas as pd
Cat = pd.read_excel(r'../De-Identified Mohs Full Data.xlsx',sheet_name='Identified Data',header=0) # Only Size at Greatest Dimension column
Cat.head()

,Randomized Study Number,Size at Greatest Dimension (S) ***Be sure this is from the note and not the Mohs episode!,Recurrent Tumor (R),"Location - Nose, Eyelid, Nail, Lip, Genitalia, or Acral (L)",Aggressive Tumor Pathology Type (AT),DFSP (D),Coordinated repair with ENT/oculoplastics (C),Additional C&F,Additional Excision,Multiple Sites,Business Days from Referral to First Appt,Physical Exam (PE),Assessment & Plan (A&P),Addendum (A)
0,36,0.8,Y,N,N,N,N,N,N,N,32.0,"Physical exam:\nGeneral: Well developed, well ...",1. Chronic actinic damage in the setting of hi...,Addendum 7/11/2023\n \nThe pathology results w...
1,240,0.6,Y,Y,N,N,N,N,N,N,40.0,Physical exam:\nGeneral: Well-dressed; well-no...,Procedure Orders\nBiopsy [486623795] ordered b...,Addendum 7/11/2023\nThe pathology results were...
2,32,0.3,N,N,N,N,N,N,N,N,4.0,Physical exam:\nGeneral: Well-dressed; well-no...,Assessment & Plan: \n1. Brown papule on the le...,Addendum 7/13/2023\n \nThe pathology results w...
3,111,6.5,N,N,N,N,Y,N,N,N,35.0,Physical exam:\nVital signs: There were no vit...,Assessment & Plan: \nBiopsy-proven basal cell ...,NaN
4,197,3.5,N,N,N,N,N,Y,Y,N,35.0,"Physical exam:\nGeneral: Well developed, well ...",Assessment & Plan: \n1. Chronic actinic sun d...,Addendum 7/5/2023\n \nThe pathology results we...


In [25]:
# Set options here
sample_size = len(Cat) # Change number to increase or decrease number of patients read from file
#sample_size = 10
multiple_columns = True # Set to True to use all input text columns, False to use only PE column

# Array of indexes containing patients with notes that have problems (Some may be ok, these are from size so a location could exist but not size)
skip_patients_array = [7,8,22,23,32,34,38,39,49,50,52,55,62,66,73,80,88,89,101,102,107,110,121,123,131,133,135,
                       136,137,140,150,155,157,164,170,184,187,193,200,206,210,214,227,228,229,234,252,261,265,279,287,292,293]
skip_patients = False # Set to True to skip bad patient notes, False to keep them in

In [26]:
# Takes the text in the PE, A&P, and A columns and asks ChatGPT to return "Yes" or "No" to the question
import re

# Always split into 3 GPT calls, one for each column
def inputSplit(input_columns, que, index):
    while len(input_columns) < 3: # Some text columns may be empty so add a blank entry
        input_columns.append("")
        
    input_string = " ".join(map(str, input_columns)) # Apply str() to each index of input_columns then join with a space
    prompt = f"{input_string} \n {que}" # Create base prompt (input_columns + question)
    
    if(len(input_string) > 0):
        tmp_ans = [""] * len(input_columns)
        check_ans = [""] * len(input_columns)
        
        # Split into 3 GPT calls, one per column, and remove anything that isn't a "Yes" or "No" from the result
        for i in range(len(input_columns)):
            prompt = f"{input_columns[i]} \n {que}" # Recreate prompt with 1 column + question
            tmp_ans[i] = call_gpt(prompt) # Call GPT with prompt and save answer to array
            
            print(f"tmp_ans[{i}] = {str(tmp_ans[i])}")
            
            check_ans[i] = re.split(r'\s+|-', tmp_ans[i])  # Split by spaces or hyphens
            check_ans[i] = [re.sub(r'[^a-zA-Z]', '', j) for j in check_ans[i]]  # Remove non-alphabetic characters
            check_ans[i] = [j.lower() for j in check_ans[i]]  # Convert all to lowercase to handle variations

            check_ans[i] = [j for j in check_ans[i] if j in ['yes', 'no']] # Keep only "yes" or "no"
            
            print(f"check_ans[{i}] = {str(check_ans[i])}")
            
        # Check if 2 arrays are the same output, if so return one of them if it is not empty
        if(check_ans[0] == check_ans[1] or check_ans[0] == check_ans[2]):
            if check_ans[0]:
                return tmp_ans[0]
            elif check_ans[1]:
                return tmp_ans[1]
            elif check_ans[2]:
                return tmp_ans[2]
        elif(check_ans[1] == check_ans[2]):
            if check_ans[1]:
                return tmp_ans[1]
            elif check_ans[2]:
                return tmp_ans[2]

        # Count num of "yes" and "no"
        yes_count = [0] * len(input_columns)
        no_count = [0] * len(input_columns)
        
        for i in range(len(check_ans)):
            yes_count[i] = check_ans[i].count('yes')
            no_count[i] = check_ans[i].count('no')
            
        print(f"Yes counts: {yes_count}")
        print(f"No counts: {no_count}")
        
        # If no matches, return string that has the most "yes" or "no"
        if any(count > 0 for count in yes_count) or any(count > 0 for count in no_count): # Check count strings for any occurences or "yes" or "no"
            max_yes_no = [0] * len(input_columns)
            max_answer = 0
            max_i = -1
            
            # Check if string has more "yes" or "no"
            for i in range(len(input_columns)):
                max_yes_no[i] = max(yes_count[i], no_count[i])
                
            # Look for highest overall "yes" or "no" then return that string
            for i in range(len(input_columns)):
                if max_answer < max_yes_no[i]:
                    max_answer = max_yes_no[i]
                    max_i = i
            return tmp_ans[max_i]
        
        # All non "yes" or "no" answers, return empty string
        return ""
    else:
        # Somehow input_string is empty
        return "input_string empty"
    

true_results = [0 for i in range(sample_size)]
true_results_final = [0 for i in range(sample_size)]

output = [0 for i in range(sample_size)]
output_final = [0 for i in range(sample_size)]

for i in range(0, min(sample_size, len(Cat))):
    # If skip_patients is True and current index is in skip_patients_array, move to next index
    if skip_patients and i in skip_patients_array:
        continue
        
    true_results[i] = Cat.at[i, 'Location - Nose, Eyelid, Nail, Lip, Genitalia, or Acral (L)']
    que = 'Question: Is there a cancerous tumor on the: Nose, Eyelid, Nail, Lip, Genitalia, or Acral? Answer with ONLY "Yes" or "No".'
    ans = ""
    
    if multiple_columns:
        input_columns = Cat.loc[i, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']]
        ans = inputSplit(input_columns, que, i) # Result from GPT call
    elif not multiple_columns:
        input_columns = Cat.at[i, 'Physical Exam (PE)']
        prompt = f"{input_columns} \n {que}"
        #if(i != 47):
            #ans = call_gpt(prompt)
    
    print(f"Result[{i}] = {ans}")
    output[i] = ans

tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No.
check_ans[1] = ['no']
tmp_ans[2] = No.
check_ans[2] = ['no']
Result[0] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = Yes
check_ans[1] = ['yes']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[1] = Yes
tmp_ans[0] = No.
check_ans[0] = ['no']
tmp_ans[1] = No.
check_ans[1] = ['no']
tmp_ans[2] = No.
check_ans[2] = ['no']
Result[2] = No.
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No.
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No.
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No.
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[5] = No
tmp_ans[0] = Yes
check_ans[0] = ['yes']
tmp_ans[1] = No.
check_ans[1] = ['no']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[6] = Yes
tmp_ans[0] = No.
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']


In [27]:
import re

# Go through every GPT response and turn it into "Y" or "N" to compare
for i in range(0, min(sample_size, len(Cat))):
    if skip_patients and i in skip_patients_array:
        continue
        
    yn = output[i]
    print(f"i={i} Answer: {yn}")
    yn_real = str(true_results[i])
    
    yn = re.split(r'\s+|-', yn)  # Split by spaces or hyphens
    yn = [re.sub(r'[^a-zA-Z]', '', j) for j in yn]  # Remove non-alphabetic characters
    yn = [j.lower() for j in yn]  # Convert all to lowercase to handle variations
    
    yn = [j for j in yn if j in ['yes', 'no']] # Keep only "yes" or "no"
    
    if len(yn) == 0:
        output_final[i] = "error_2"  # No valid response detected, bad prediction
    
    yes_count = yn.count('yes')
    no_count = yn.count('no')
    
    if yes_count > no_count:
        output_final[i] = "Y"
    elif no_count > yes_count:
        output_final[i] = "N"
        
    # Check if the given input is valid
    if yn_real.strip():
        true_results_final[i] = yn_real
    else:
        true_results_final[i] = "error_1" # No valid input provided in true_results, bad note
    
    print(f"Result[{i}]: ChatGPT Prediction = {output_final[i]} Answer Given = {true_results_final[i]}")

i=0 Answer: No
Result[0]: ChatGPT Prediction = N Answer Given = N
i=1 Answer: Yes
Result[1]: ChatGPT Prediction = Y Answer Given = Y
i=2 Answer: No.
Result[2]: ChatGPT Prediction = N Answer Given = N
i=3 Answer: No
Result[3]: ChatGPT Prediction = N Answer Given = N
i=4 Answer: No
Result[4]: ChatGPT Prediction = N Answer Given = N
i=5 Answer: No
Result[5]: ChatGPT Prediction = N Answer Given = N
i=6 Answer: Yes
Result[6]: ChatGPT Prediction = Y Answer Given = Y
i=7 Answer: No.
Result[7]: ChatGPT Prediction = N Answer Given = N
i=8 Answer: No
Result[8]: ChatGPT Prediction = N Answer Given = N
i=9 Answer: No.
Result[9]: ChatGPT Prediction = N Answer Given = N
i=10 Answer: No
Result[10]: ChatGPT Prediction = N Answer Given = N
i=11 Answer: No
Result[11]: ChatGPT Prediction = N Answer Given = N
i=12 Answer: No.
Result[12]: ChatGPT Prediction = N Answer Given = N
i=13 Answer: Yes
Result[13]: ChatGPT Prediction = Y Answer Given = Y
i=14 Answer: No.
Result[14]: ChatGPT Prediction = N Answer Gi

In [31]:
total_size = 0
correct_predictions = 0
no_response_given = 0
no_alphabetical_result_returned = 0
bad_cases_combined = 0
nrg_not_narr = 0
narr_not_nrg = 0
# nrg = no valid response given in (L) column
# narr = no alphabetical result returned from ChatGPT output

for i in range(0, min((sample_size), len(Cat))):
    if(output_final[i] == "error_2" and true_results_final[i] == "error_1"):
        bad_cases_combined += 1
        
    if(output_final[i] == "error_2"):
        no_alphabetical_result_returned += 1
        if(true_results_final[i] != "error_1"):
            narr_not_nrg += 1
            
    if(true_results_final[i] == "error_1"):
        no_response_given += 1
        if(output_final[i] != "error_2"):
            nrg_not_narr += 1
            
    if(output_final[i] == true_results_final[i]):
        correct_predictions += 1
        
    total_size += 1
    
accuracy = correct_predictions / total_size
print("# of Correct Predictions = " + str(correct_predictions))
print("Sample Size = " + str(total_size))
print("Accuracy = " + "{:.2f}%".format(accuracy * 100))

# Accuracy when removing cases where the Location is not given (best to look at)
no_response_given_accuracy = correct_predictions / (total_size - no_response_given)
print()
print("Number of Notes without a Given Location = " + str(no_response_given))
print("Accuracy = " +"{:.2f}%".format(no_response_given_accuracy * 100))

# Accuracy when removing cases where GPT didn't return a valid alphabetical answer (not the best to look at, could be slightly lower)
no_alphabetical_result_returned_accuracy = correct_predictions / (total_size - no_alphabetical_result_returned)
print()
print("Number of PE/A&P/A Notes without a valid GPT Given Location = " + str(no_alphabetical_result_returned))
print("Accuracy = " + "{:.2f}%".format(no_alphabetical_result_returned_accuracy * 100))

# Accuracy when removing all above bad cases (ok to look at, could be slightly lower)
bad_cases_total_accuracy = correct_predictions / (total_size - (narr_not_nrg + nrg_not_narr + bad_cases_combined))
print()
print("Number of Combined Notes without a Given Location = " + str(narr_not_nrg + nrg_not_narr + bad_cases_combined))
print("Accuracy = " + "{:.2f}%".format(bad_cases_total_accuracy * 100))     

# of Correct Predictions = 237
Sample Size = 294
Accuracy = 80.61%

Number of Notes without a Given Location = 0
Accuracy = 80.61%

Number of PE/A&P/A Notes without a valid GPT Given Location = 0
Accuracy = 80.61%

Number of Combined Notes without a Given Location = 0
Accuracy = 80.61%
